## Imports

In [2]:
import pandas as pd
import numpy as np
import os
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
import json

## Define peak hours

In [7]:
# Define peak hours 8-11 and 16-19
peak_hours = list(range(8,19))
# All hours
all_hours = list(range(24))
# difference between peak hours and all hours (off_peak_hours)
off_peak_hours = list(set(all_hours) - set(peak_hours))

## Spot prices from the dataset
Load and prepare dataset

In [9]:
# load dataset
fp = "data/gridtx-dump-AGGREGATED-CLEANED-THRESHOLD-COVERAGE100-NORMALIZED.csv"
data = pd.read_csv(fp, sep=",")

# Meter with most observations
mm_id = "28ba7f57-6e83-4341-8078-232c1639e4e3"

# df = data[data["meter_id"] == mm_id]
df = data

df_cons = df[df["type"] == "consumption"]

# Convert timeslot to datetime and set as index
df_cons.index = pd.to_datetime(df_cons['timeslot'], utc=True)

# Sort index 
df_cons = df_cons.sort_index()
df_cons["hour"] = df_cons.index.hour
# Add column to df_cons_spot_price
df_cons["peak_hour"] = np.where(df_cons["hour"].isin(peak_hours), 1, 0)

Calculate averages

In [10]:
#Average over peak_hour
df_cons_avg = df_cons.groupby(["peak_hour"]).mean()["spot_price_no_vat"]
avg_dict = df_cons_avg.to_dict()

# Average over hour
df_cons_avg_hour = df_cons.groupby(["hour"]).mean()["spot_price_no_vat"]
df_cons_avg_hour_dict = df_cons_avg_hour.to_dict()

hour_lookup_price_dict = {}
for hour in all_hours:
    if hour in peak_hours:
        hour_lookup_price_dict[hour] = avg_dict[1]
    elif hour in off_peak_hours:
        hour_lookup_price_dict[hour] = avg_dict[0]

Save to files

In [ ]:
# Save to files
with open('data/hour_lookup_price_dict.json', 'w') as fp:
    json.dump(hour_lookup_price_dict, fp)

with open('data/hour_lookup_price_houravg_dict.json', 'w') as fp:
    json.dump(df_cons_avg_hour_dict, fp)

## Spot prices from Nordpool (2021)
Load and prepare data


In [14]:
# load spot_prices_2021.csv
fp = "data/spot_prices_2021.csv"
df = pd.read_csv(fp, sep=",")

df_oslo = df[df["region"] == "Oslo"]

# Convert timeslot to datetime and set as index
df_oslo.index = pd.to_datetime(df_oslo['start_time'], utc=True)

# Sort index 
df_oslo = df_oslo.sort_index()
df_oslo["hour"] = df_oslo.index.hour
df_oslo["peak_hour"] = np.where(df_oslo["hour"].isin(peak_hours), 1, 0)


Calculate averages

In [ ]:
#Average over peak_hour
df_oslo_avg_peak_hour = df_oslo.groupby(["peak_hour"]).mean()["price"]
df_oslo_avg_hour = df_oslo.groupby(["hour"]).mean()["price"]
avg_dict_oslo_hour = df_oslo_avg_hour.to_dict()

# df_cons_avg to dict
avg_peak_hour_dict = df_oslo_avg_peak_hour.to_dict()

hour_lookup_price_dict = {}
for hour in all_hours:
    if hour in peak_hours:
        hour_lookup_price_dict[hour] = avg_peak_hour_dict[1]
    elif hour in off_peak_hours:
        hour_lookup_price_dict[hour] = avg_peak_hour_dict[0]

Save to files

In [ ]:
with open('data/hour_lookup_price_oslo_2021_dict.json', 'w') as fp:
    json.dump(hour_lookup_price_dict, fp)

with open('data/hour_lookup_price_oslo_2021_houravg_dict.json', 'w') as fp:
    json.dump(avg_dict_oslo_hour, fp)